In [3]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json

In [27]:
tags = [
    'malaysia',
    'singapore',
    'money',
    'world',
    'life',
    'eat-drink',
    'showbiz',
    'opinion',
    'sports',
    'tech-gadgets',
    'what-you-think'
]

In [29]:
directory = 'url'
!mkdir {directory}

In [50]:
def crawl(url, filename):
    if os.path.exists(filename):
        return
    while True:
        try:
            r = requests.get(url)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    news_urls = list(set([a_ for a_ in a if url_ in a_]))
    with open(filename, 'w') as fopen:
        json.dump(news_urls, fopen)
        
    return news_urls

In [40]:
urls[0]

('https://www.malaymail.com/morearticles/malaysia?pgno=5131',
 'url/malaysia-5131.json')

In [51]:
from concurrent.futures import ThreadPoolExecutor, as_completed

max_worker = 10

for t in tags:
    print(t)
    url = f'https://www.malaymail.com/morearticles/{t}'
    url_ = url.replace('/morearticles/', '/news/')
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    pgno = [a_ for a_ in a if 'pgno' in a_]
    max_page = max([int(a_.split('pgno=')[1]) for a_ in pgno])
    
    for i in tqdm(range(1, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [(f'{url}?pgno={a}', os.path.join(directory, f'{t}-{a}.json')) for a in aranged]
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url[0], url[1]): url for url in urls}
        
        for future in as_completed(futures):
            future.result()

malaysia


100%|███████████████████████████████████████| 905/905 [00:00<00:00, 3508.98it/s]


singapore


100%|█████████████████████████████████████████| 25/25 [00:00<00:00, 2253.16it/s]


money


...

100%|█████████████████████████████████████████| 426/426 [19:20<00:00,  2.72s/it]


tech-gadgets


100%|███████████████████████████████████████████| 88/88 [02:55<00:00,  1.99s/it]


what-you-think


100%|███████████████████████████████████████████| 54/54 [01:44<00:00,  1.93s/it]
